# Transformer-based Natural Language Processing
## Introduction to 🤗 Transformers
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/texttechnologylab/WiSe22-M-PNLR-PR-TbNLP/blob/master/transformers.ipynb)

### Installing necessary packages (i.e. if on Colab)

In [ ]:
% pip install torch datasets tokenizers transformers

### Premise

This notebook will guide you through the process of finetuning a transformer model using the 🤗 Transformers library.

First, we need to select a task and suitable dataset. Here, we will use the [Textual Entailment](https://nlp.stanford.edu/projects/snli/) task as an example. A suitable dataset can be found in the [`snli` repository on the 🤗 Dataset hub](https://huggingface.co/datasets/snli).

Thus, we can load the Recognizing Textual Entailment subset of the SuperGLUE dataset as follows:

In [1]:
from datasets import load_dataset

from datasets import disable_caching
disable_caching()

snli_dataset = load_dataset("snli")
print(snli_dataset)

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 550152
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
})


As we can see above, the dataset is already split into train, development and test splits.
Each row contains four, but we only need to focus the premise, hypothesis and the label.

The textual entailment task requires us to recognize, given two text fragments, whether the meaning of one text is entailed (*can be inferred*) from the other text.

In this example, we will use a BERT-family model. With BERT, we formulate the entailment task as a simple classification task by concatenating the premise and hypothesis and training our classifier on the first token (the `[CLS]` token) of the input string:

```
"[CLS] This is the premise, i.e. a text that means something. [SEP] This is the hypothesis, i.e. what we may be able to infer [SEP]"
```

But let's first take a look at the dataset.

In [2]:
print(snli_dataset['train'][:2])
print(snli_dataset['validation'][:2])
print(snli_dataset['test'][:2])

{'premise': ['A person on a horse jumps over a broken down airplane.', 'A person on a horse jumps over a broken down airplane.'], 'hypothesis': ['A person is training his horse for a competition.', 'A person is at a diner, ordering an omelette.'], 'label': [1, 2]}
{'premise': ['Two women are embracing while holding to go packages.', 'Two women are embracing while holding to go packages.'], 'hypothesis': ['The sisters are hugging goodbye while holding to go packages after just eating lunch.', 'Two woman are holding packages.'], 'label': [1, 0]}
{'premise': ['This church choir sings to the masses as they sing joyous songs from the book at a church.', 'This church choir sings to the masses as they sing joyous songs from the book at a church.'], 'hypothesis': ['The church has cracks in the ceiling.', 'The church is filled with song.'], 'label': [1, 0]}


As we see above, the `test` split contains **unlabeled** samples, be we can ignore that for now.

Let's construct the sentences as we outlined above.

In [4]:
prepared_dataset = snli_dataset.filter(lambda sample: sample['label'] >= 0)
prepared_dataset = prepared_dataset.map(
    lambda sample: {'input': f"{sample['premise']} [SEP] {sample['hypothesis']}"},
    remove_columns=['premise', 'hypothesis']
)
print(prepared_dataset['train'][:3])

{'label': [1, 2, 0], 'input': ['A person on a horse jumps over a broken down airplane. [SEP] A person is training his horse for a competition.', 'A person on a horse jumps over a broken down airplane. [SEP] A person is at a diner, ordering an omelette.', 'A person on a horse jumps over a broken down airplane. [SEP] A person is outdoors, on a horse.']}


### Loading Pre-Trained Models

Now we need to load a pre-trained BERT model. You should use a subclass of [AutoModel](https://huggingface.co/docs/transformers/main/en/autoclass_tutorial).

#### Load and instantiate a model for the textual entailment task

In [ ]:
from transformers import AutoConfig, AutoTokenizer  #, AutoModelFor?

config = ...  # TODO
tokenizer = ...  # TODO
model = ...  # TODO

Now we could use the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) class for easy training. You can follow the tutorial from [the official documentation](https://huggingface.co/docs/transformers/quicktour#trainer-a-pytorch-optimized-training-loop).

#### Write the training procedure

In [ ]:
from transformers import Trainer

trainer = Trainer(...)

# TODO

trainer.train()

# TODO: evaluation

### Custom Training
While using the trainer class is very convenient, if you have to run custom procedures during training, a regular training loop can be more accessible.

We can re-use code from the datasets notebook.

In [ ]:
def encode_pt(batch: dict):
    return tokenizer(
        batch['input'],
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=False,
        padding=False,
        truncation=True,
    )


pt_dataset = prepared_dataset.map(encode_pt)
print(pt_dataset['train'][:2])

However, in a manual training loop, we will want to make use of PyTorch's DataLoaders, which require some extra care to collate batches with samples of different lengths.

#### Implement `custom_collate`:
- Pad and stack the `input_ids` in a tensor.
- Stack the labels in a tensor of type `long`.

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence


def custom_collate(batch: list[dict]) -> tuple[torch.Tensor, torch.Tensor]:
    input_ids = ...
    label = ...
    return input_ids, label

#### Write the training and evaluation loops

In [ ]:
from tqdm.notebook import tqdm, trange
from torch.utils.data import DataLoader
from torch.optim import *  # TODO
from torch.nn import *  # TODO

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

criterion = ...  # TODO
optimizer = ...  # TODO
num_epochs = ...  # TODO
batch_size = ...  # TODO

train_dataloader = DataLoader(pt_dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
dev_dataloader = DataLoader(pt_dataset['validation'], batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

model.to(device)
for epoch in trange(num_epochs, position=0):
    model.train()
    for batch in tqdm(train_dataloader, position=1, leave=False):
        ...  # TODO

    model.eval()
    for batch in tqdm(dev_dataloader, position=1, leave=False):
        ...  # TODO